# Welcome to Flyway!

This notebook walks you through a tutorial which introduces you to Flyway concepts and commands. The tutorial simulates beginning to use an existing production database with Flyway.

# Setup

## A) Set up Flyway: install, configure connection for development, set location of migrations

```
1. Download and extract Flyway for your platform - [https://flywaydb.org/download/](https://flywaydb.org/download/)
1. Edit <install-dir>/conf/flyway.conf to connect to your development database
    * I set: *flyway.url=jdbc:postgresql://pg-demo-dev.postgres.database.azure.com/postgres*
    * I also set defaults for *flyway.user* (username@gpt-demo-dev) and *flyway.password* arguments
    * I set: *flyway.locations=filesystem:C:\Git\Flyway\migrations* 
    * I set: *flyway.licensekey*=<trial license key received by email>
1. Save a copy of flyway.conf in <install-dir> to connect to your production database
    * Modify the flyway.url, flyway.user, and flyway.password argument values to connect to "production"
1. Set an environment variable: FLYWAY_EDITION should be set to pro or Enterprise (this is required for the 'undo' portion of the demo)
```

## B) Setup postgres databases

To run through this demo, you will need two postgres databases.

I created two postgres databases hosted in Azure. 

* pg-demo-dev.postgres.database.azure.com
* pg-demo-prod.postgres.database.azure.com

I then went to 'Connection Security' for each database and configured an exception for my IP Address so I can connect to each from Azure Data Studio on my desktop.

*Database names are provided here simply to help make it clear what is happening in the demo. You will need your own databases to follow along. They may be hosted using the method of your choice-- Azure isn't required.*

### Set up the 'production' database

Connect to pg-demo-prod. This is meant to represent an existing production database, so we are going to create some tables that will act like an existing production environment.




In [2]:
/* These tables are built from an older version of the Stack Exchange Data Dump: https://archive.org/details/stackexchange

The code has been adapted to PostgreSQL's SQL.

Stack Overflow's TSQL is shared under Attribution-ShareAlike 4.0 International (CC BY-SA 4.0) https://creativecommons.org/licenses/by-sa/4.0/

*/

CREATE TABLE Badges (
	Id int GENERATED BY DEFAULT AS IDENTITY NOT NULL UNIQUE PRIMARY KEY,
	Name varchar(40) NOT NULL,
	UserId int NOT NULL,
	Date timestamp(0) NOT NULL
);


CREATE TABLE Comments (
	Id int GENERATED BY DEFAULT AS IDENTITY NOT NULL UNIQUE PRIMARY KEY,
	CreationDate timestamp(0) NOT NULL,
	PostId int NOT NULL,
	Score int NULL,
	Text varchar(700) NOT NULL,
	UserId int NULL
);

CREATE TABLE LinkTypes (
	Id int GENERATED BY DEFAULT AS IDENTITY NOT NULL UNIQUE PRIMARY KEY,
	Type varchar(50) NOT NULL
);

CREATE TABLE PostLinks (
	Id int GENERATED BY DEFAULT AS IDENTITY NOT NULL UNIQUE PRIMARY KEY,
	CreationDate timestamp(0) NOT NULL,
	PostId int NOT NULL,
	RelatedPostId int NOT NULL,
	LinkTypeId int NOT NULL
);

CREATE TABLE Posts (
	Id int GENERATED BY DEFAULT AS IDENTITY NOT NULL UNIQUE PRIMARY KEY,
	AcceptedAnswerId int NULL,
	AnswerCount int NULL,
	Body text NOT NULL,
	ClosedDate timestamp(0) NULL,
	CommentCount int NULL,
	CommunityOwnedDate timestamp(0) NULL,
	CreationDate timestamp(0) NOT NULL,
	FavoriteCount int NULL,
	LastActivityDate timestamp(0) NOT NULL,
	LastEditDate timestamp(0) NULL,
	LastEditorDisplayName varchar(40) NULL,
	LastEditorUserId int NULL,
	OwnerUserId int NULL,
	ParentId int NULL,
	PostTypeId int NOT NULL,
	Score int NOT NULL,
	Tags varchar(150) NULL,
	Title varchar(250) NULL,
	ViewCount int NOT NULL
);


CREATE TABLE PostTypes (
	Id int GENERATED BY DEFAULT AS IDENTITY NOT NULL UNIQUE PRIMARY KEY,
	Type varchar(50) NOT NULL
);

CREATE TABLE Users (
	Id int GENERATED BY DEFAULT AS IDENTITY NOT NULL UNIQUE PRIMARY KEY,
	AboutMe text NULL,
	Age int NULL,
	CreationDate timestamp(0) NOT NULL,
	DisplayName varchar(40) NOT NULL,
	DownVotes int NOT NULL,
	EmailHash varchar(40) NULL,
	LastAccessDate timestamp(0) NOT NULL,
	Location varchar(100) NULL,
	Reputation int NOT NULL,
	UpVotes int NOT NULL,
	Views int NOT NULL,
	WebsiteUrl varchar(200) NULL,
	AccountId int NULL
);

CREATE TABLE Votes (
	Id int GENERATED BY DEFAULT AS IDENTITY NOT NULL UNIQUE PRIMARY KEY,
	PostId int NOT NULL,
	UserId int NULL,
	BountyAmount int NULL,
	VoteTypeId int NOT NULL,
	CreationDate timestamp(0) NOT NULL
);

CREATE TABLE VoteTypes (
	Id int GENERATED BY DEFAULT AS IDENTITY NOT NULL UNIQUE PRIMARY KEY,
	Name varchar(50) NOT NULL
);

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:01.112

### Query the development database

Connect to pg-demo-dev and run a couple of queries.

In [3]:
SELECT version(), current_database();

(1 row(s) affected)

Total execution time: 00:00:00.483

version,current_database
"PostgreSQL 10.11, compiled by Visual C++ build 1800, 64-bit",postgres


In [4]:
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema not in ( 'pg_catalog' , 'information_schema' );

(2 row(s) affected)

Total execution time: 00:00:00.474

table_schema,table_name
public,pg_stat_statements
public,pg_buffercache


# 2) Learn Flyway commands 

## Run commands against the development database: info, migrate, clean

From a terminal:

    flyway info

[https://flywaydb.org/documentation/commandline/info](https://flywaydb.org/documentation/commandline/info)

Take a look at the file C:\Git\Flyway\migrations\V1__Base_version.sql

* This is a file containing a baseline version
* For an existing database, this is a script for the schema you are starting with
* Having this script allows you to repeatedly test all your migrations against a "clean" database 

Next run:

    flyway migrate

[https://flywaydb.org/documentation/commandline/migrate](https://flywaydb.org/documentation/commandline/migrate)


Now run again:

    flyway info

Next, we are going to delete everything in the development database by running a 'clean'. Note: **be careful where you clean**!

    flyway clean

Now use 'flyway info' and 'flyway migrate' again to review the commands.

## Query the development database to find migrations (so far we only have the baseline migration)

In [5]:
SELECT  installed_rank, version, description, type, script, checksum, installed_by, installed_on, execution_time, success
FROM public.flyway_schema_history
ORDER BY execution_time DESC;

(1 row(s) affected)

Total execution time: 00:00:00.475

installed_rank,version,description,type,script,checksum,installed_by,installed_on,execution_time,success
1,1,Base version,SQL,V1__Base_version.sql,-1488644070,kendar,2020-04-28T16:10:20.761557,1412,True


# 3) Create a migration with an undo script and test against development

Copy both files from the part_3_of_demo folder into the migrations folder. Review the files:

* U1.1__Add_new_table.sql
* V1.1__Add_new_table.sql

Run the 'up' migration against the development database:
    
    flyway migrate

Review migrations which have been run:

    flyway info

Run the 'undo' migration:

    flyway undo

[https://flywaydb.org/documentation/command/undo](https://flywaydb.org/documentation/command/undo)

Review migrations which have been run:

    flyway info

Redo the 'migrate' and 'info' commands and review.

# 4) Baseline production, then run the 'up' migration against production

Run flyway info against 'production':

    flyway -configFiles="C:\flyway\flyway-6.4.0\conf\flyway.prod.conf" info


Try to migrate. What happens?

    flyway -configFiles="C:\flyway\flyway-6.4.0\conf\flyway.prod.conf" migrate

We need to baseline:

    flyway -configFiles="C:\flyway\flyway-6.4.0\conf\flyway.prod.conf" baseline

[https://flywaydb.org/documentation/command/baseline](https://flywaydb.org/documentation/command/baseline)

Check the status"

    flyway -configFiles="C:\flyway\flyway-6.4.0\conf\flyway.prod.conf" info

Now we migrate:

    flyway -configFiles="C:\flyway\flyway-6.4.0\conf\flyway.prod.conf" migrate


Check info, review the new table in the database, and optionally run undo if you want to roll back.

# 5) Reset the demo

If you'd like to reset your demo environment so that you can run through this tutorial again, here is how:

Clean the 'production' database. In the real world you would NEVER want to clean production, it **drops everything**. But in this demo case, we have a script in the setup which creates the "production" environment.

    flyway -configFiles="C:\flyway\flyway-6.4.0\conf\flyway.prod.conf" clean

Remove files from your migrations directory. You should have these files stil in the "part_3_of_demo" directory for the next run through.

* U2.1__Add_new_table.sql
* V2.1__Add_new_table.sql

Run a clean against the dev enironment

    flyway clean